In [1]:
import os
import glob
import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from scipy import stats
import pandas as pd
import bokeh_catplot
import math

from bokeh.layouts import row
bokeh.io.output_notebook()

Loading BokehJS ...

In [9]:
#Functions
def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians

def getDataFrame(csv_file, tot_pili_limit):
    df = pd.read_csv(csv_file, sep=',', na_values='*')
    df['PoleRange']=df['AreaPoleDim']/df['CellArea']
    pole_range=0.15 #df.PoleRange.mean()
    print(pole_range)
    ref_ratio = (2*pole_range)/(1-2*pole_range)
    df['PercentTotalFluoDim']=df['TotalFluorescencePoleDim']/df['CellTotalFluorescence']
    df['PercentTotalFluoBright']=df['TotalFluorescencePoleBright']/df['CellTotalFluorescence']
    df['PercentTotalFluoCenter']=(df['CellTotalFluorescence']-df['TotalFluorescencePoleBright']-df['TotalFluorescencePoleDim'])/df['CellTotalFluorescence']
    #df['polar_ratio']=(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright'])/(df['CellTotalFluorescence']-(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright']))/((2*df['PoleRange']/(1-2*df['PoleRange'])))
    df['polar_ratio']=((df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright'])/(df['CellTotalFluorescence']-(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright'])))/ref_ratio
    df['TotalPili']=df['Nb_Pili_PoleDim']+df['Nb_Pili_PoleBright']
    df['TotalFlagella']=df['Nb_Flagella_PoleDim']+df['Nb_Flagella_PoleBright']
    y_param1='Nb_Pili_PoleDim'
    y_param2='Nb_Pili_PoleBright'
    x_param1='TotalFluorescencePoleDim'
    x_param2='TotalFluorescencePoleBright'
    parameterX='TotalPili'
    parameterY='PoleTotalFluorescence'
    poles_pili_dim=list(df[y_param1])
    poles_pili_bright=list(df[y_param2])
    poles_pili=poles_pili_dim+poles_pili_bright
    poles_pili=np.asarray(poles_pili).reshape(len(poles_pili),1)
    poles_fluo_dim=list(df[x_param1])
    poles_fluo_bright=list(df[x_param2])
    poles_fluo=poles_fluo_dim+poles_fluo_bright
    poles_fluo=np.asarray(poles_fluo).reshape(len(poles_fluo),1)
    names = [parameterX, parameterY]
    data = np.concatenate((poles_pili, poles_fluo), axis=1)
    df_long=pd.DataFrame(data=np.transpose(data), index=names).T
    y_param1='Nb_Pili_PoleDim'
    y_param2='Nb_Pili_PoleBright'
    Ncells_per_pili_Dim=np.empty((tot_pili_limit), dtype='int16')
    Ncells_per_pili_Bright=np.empty((tot_pili_limit), dtype='int16')
    N=len(df)
    print(df.Strain.unique()[0])
    print('Dim pole:')
    print('NCells='+str(N), end=', ')
    for i in range(tot_pili_limit):
        Ncells_per_pili_Dim[i]=len(df.loc[(df[y_param1]==i)])
        if (i<tot_pili_limit-1):
            print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]), end=', ')
        else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]))
    print('Bright pole:')
    print('Ncells='+str(N), end=', ')
    for i in range(tot_pili_limit):
        Ncells_per_pili_Bright[i]=len(df.loc[(df[y_param2]==i)])
        if (i<tot_pili_limit-1):
            print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]), end=', ')
        else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]))
    nb_pili=np.zeros((tot_pili_limit, 1))
    boot_mean=np.zeros((tot_pili_limit, 1))
    boot_median=np.zeros((tot_pili_limit, 1))
    boot_IC_mean=np.zeros((tot_pili_limit, 2))
    boot_IC_median=np.zeros((tot_pili_limit, 2))
    for n_pili in range(tot_pili_limit):
        Pole_fluorescence_array=df_long.PoleTotalFluorescence.loc[(df_long[parameterX]==n_pili)].values
        nb_pili[n_pili]=n_pili
        if (len(Pole_fluorescence_array) > 0):
            bs_fluorescence_array=bootstrap_sampling(Pole_fluorescence_array, len(Pole_fluorescence_array), 1000)
            [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_fluorescence_array)
            boot_mean[n_pili]=bs_means
            boot_IC_mean[n_pili,:]=IC_means
            boot_median[n_pili]=bs_medians
            boot_IC_median[n_pili,:]=IC_medians
    names = ['TotalPili', 'bootMean','bootMedian']
    data = np.concatenate((nb_pili, boot_mean, boot_median), axis=1)
    df_boot_stats_tot=pd.DataFrame(data=np.transpose(data), index=names).T
    df_boot_stats_tot['IC_mean']=list(boot_IC_mean)
    df_boot_stats_tot['IC_median']=list(boot_IC_median)
    return df, df_long, df_boot_stats_tot

def plotData(df, df_long, df_boot_stats_tot, Xrange, Yrange, parameterX, parameterY, param1, param2, IC1):
    p3 = bokeh.plotting.figure(
        width=600, 
        height=600, 
        x_axis_label='# pili', 
        y_axis_type='linear',
        y_range=Yrange,
        x_range=Xrange,
        y_axis_label ='pole fluorescence',
        title=str(df.Strain.unique()[0])+" pole total fluorescence vs # pili (bootstrap median, 95% CI)"
    )
    p3.circle(
        source=df_long,
        x=parameterX,
        y=parameterY, 
        line_color = 'green',
        fill_color = 'white',
        alpha=0.4,
        #legend = labelsAll[i]
    )
    for n_pili in list(df_boot_stats_tot[param2]):
        b = [n_pili,n_pili]
        ic=df_boot_stats_tot.loc[(df_boot_stats_tot[param2] == n_pili), [IC1]].values[0][0]
        p3.line(
            x = b,
            y = ic,
            color = 'black',
            alpha=0.7,
            line_width=3
        )
    #p3.line(
     #   source = df_boot_stats_tot.loc[:, [param2, param1]],
      #  x = param2,
       # y = param1,
       # line_color = 'black',
      #  alpha=0.6
    #)
    p3.circle(
        source = df_boot_stats_tot.loc[:, [param2, param1]],
        x = param2,
        y = param1,
        line_color = 'black',
        fill_color = 'white',
        alpha=0.6,
        size=10
    )
    p3.output_backend = 'svg'
    return p3

def plotPolarRatios(df_pilH, parameter):
    XRANGE=['Liquid', 'Solid']
    df_pilH.loc[df_pilH['Growth']=='Liquid', 'X_values']=0.5
    df_pilH.loc[df_pilH['Growth']=='Solid', 'X_values']=1.5
    liq_array=df_pilH[parameter].loc[(df_pilH['Growth']=='Liquid')].values
    bs_liq_array=bootstrap_sampling(liq_array, len(liq_array), 1000)
    [bs_means_liq, bs_medians_liq, IC_means_liq, IC_medians_liq]=bootstrap_stats(bs_liq_array)
    sol_array=df_pilH[parameter].loc[(df_pilH['Growth']=='Solid')].values
    bs_sol_array=bootstrap_sampling(sol_array, len(sol_array), 1000)
    [bs_means_sol, bs_medians_sol, IC_means_sol, IC_medians_sol]=bootstrap_stats(bs_sol_array)
    p3 = bokeh.plotting.figure(
        width=400, 
        height=400, 
        y_axis_type='linear',
        y_range=[0,2],
        x_range=XRANGE,
        y_axis_label ='Polarization Index',
        title=str(df_pilH.Strain.unique()[0])+" Polarization Index (bootstrap median, 95% CI)"
    )
    p3.circle(
        source=df_pilH.loc[df_pilH['Growth']=='Liquid'],
        x=jitter('X_values', width=0.3, range=p3.y_range),
        y=parameter, 
        line_color = 'darkgrey',
        fill_color = 'darkgrey',
        alpha=0.8,
        #legend = labelsAll[i]
    )
    b = ['Liquid','Liquid']
    ic=IC_medians_liq[0] 
    p3.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=1,
        line_width=4
    )
    p3.circle(
        x=0.5,
        y=bs_medians_liq, 
        line_color = 'black',
        fill_color = 'white',
        alpha=1,
        size=10
        #legend = labelsAll[i]
    )
    p3.circle(
        source=df_pilH.loc[df_pilH['Growth']=='Solid'],
        x=jitter('X_values', width=0.3, range=p3.y_range),
        y=parameter, 
        line_color = 'grey',
        fill_color = 'grey',
        alpha=0.8,
        #legend = labelsAll[i]
    )
    b = ['Solid','Solid']
    ic=IC_medians_sol[0] 
    p3.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=1,
        line_width=4
    )
    p3.circle(
        x=1.5,
        y=bs_medians_sol, 
        line_color = 'black',
        fill_color = 'white',
        alpha=1,
        size=10
        #legend = labelsAll[i]
    )
    #p3.xaxis.major_label_orientation = math.pi/4
    p3.xaxis.major_label_text_font_size = "14pt"
    p3.output_backend = 'svg'
    return p3

In [10]:
data_path="C:/users/tala/Desktop/git/PhD_codes/Mechanosensation/Python_code/Pole_analysis/Combined_Data"
os.chdir(data_path)
extension = 'csv'
list_csv = []
for root, dirs, files in os.walk(data_path, topdown=False):
    for name in files:
        if extension in name:
            list_csv.append(name)
tot_pili_limit=8
parameterX='TotalPili'
parameterY='PoleTotalFluorescence'
df_pilB, df_long_pilB, df_boot_stats_tot_pilB= getDataFrame('PolesData_pilB_mNG_fliC-_combined.csv', tot_pili_limit)
print('')
df_pilH, df_long_pilH, df_boot_stats_tot_pilH= getDataFrame('PolesData_mNG_pilH_fliC-_combined.csv', tot_pili_limit)
print('')
df_pilT, df_long_pilT, df_boot_stats_tot_pilT= getDataFrame('PolesData_mNG_pilT_fliC-_combined.csv', tot_pili_limit)
print('')
df_pilU, df_long_pilU, df_boot_stats_tot_pilU= getDataFrame('PolesData_mNG_pilU_fliC-_combined.csv', tot_pili_limit)

0.15
pilB_mNG_fliC-
Dim pole:
NCells=120, N_0pili=83, N_1pili=28, N_2pili=6, N_3pili=2, N_4pili=0, N_5pili=1, N_6pili=0, N_7pili=0
Bright pole:
Ncells=120, N_0pili=66, N_1pili=38, N_2pili=8, N_3pili=4, N_4pili=0, N_5pili=2, N_6pili=1, N_7pili=0

0.15
mNG_pilH_fliC-
Dim pole:
NCells=97, N_0pili=50, N_1pili=14, N_2pili=14, N_3pili=6, N_4pili=6, N_5pili=4, N_6pili=2, N_7pili=0
Bright pole:
Ncells=97, N_0pili=38, N_1pili=13, N_2pili=9, N_3pili=10, N_4pili=10, N_5pili=6, N_6pili=5, N_7pili=1

0.15
mNG_pilT_fliC-
Dim pole:
NCells=135, N_0pili=80, N_1pili=26, N_2pili=13, N_3pili=7, N_4pili=4, N_5pili=3, N_6pili=1, N_7pili=0
Bright pole:
Ncells=135, N_0pili=78, N_1pili=30, N_2pili=14, N_3pili=8, N_4pili=4, N_5pili=0, N_6pili=0, N_7pili=1

0.15
mNG_pilU_fliC-
Dim pole:
NCells=114, N_0pili=56, N_1pili=36, N_2pili=16, N_3pili=4, N_4pili=0, N_5pili=2, N_6pili=0, N_7pili=0
Bright pole:
Ncells=114, N_0pili=54, N_1pili=25, N_2pili=19, N_3pili=7, N_4pili=6, N_5pili=1, N_6pili=1, N_7pili=0


In [11]:
param2='TotalPili'
IC1='IC_median'
param1='bootMedian'
Yrange=[0, 100000]
Xrange=[-0.6, tot_pili_limit-2+0.6]
p_pilB=plotData(df_pilB, df_long_pilB, df_boot_stats_tot_pilB, Xrange, Yrange, parameterX, parameterY, param1, param2, IC1)
p_pilH=plotData(df_pilH, df_long_pilH, df_boot_stats_tot_pilH, Xrange, Yrange, parameterX, parameterY, param1, param2, IC1)
p_pilT=plotData(df_pilT, df_long_pilT, df_boot_stats_tot_pilT, Xrange, Yrange, parameterX, parameterY, param1, param2, IC1)
p_pilU=plotData(df_pilU, df_long_pilU, df_boot_stats_tot_pilU, Xrange, Yrange, parameterX, parameterY, param1, param2, IC1)
bokeh.io.show(row(p_pilB, p_pilH))

In [12]:
bokeh.io.show(row(p_pilT, p_pilU))

In [13]:
parameter = 'PolarRatio'
p_rat_pilH=plotPolarRatios(df_pilH, parameter)
p_rat_pilB=plotPolarRatios(df_pilB, parameter)
p_rat_pilT=plotPolarRatios(df_pilT, parameter)
p_rat_pilU=plotPolarRatios(df_pilU, parameter)
bokeh.io.show(row(p_rat_pilH, p_rat_pilB, p_rat_pilT, p_rat_pilU))

In [14]:
parameter = 'polar_ratio'
p_rat_pilH=plotPolarRatios(df_pilH, parameter)
p_rat_pilB=plotPolarRatios(df_pilB, parameter)
p_rat_pilT=plotPolarRatios(df_pilT, parameter)
p_rat_pilU=plotPolarRatios(df_pilU, parameter)
bokeh.io.show(row(p_rat_pilH, p_rat_pilB, p_rat_pilT, p_rat_pilU))